In [1]:
from clickhouse_driver import Client
from datetime import datetime
from matplotlib.dates import relativedelta
from oversea_hwc.pandas_sql import GetFromDuckdb, DateGenerator
from header_files.func import ck_read,  warehouse_info,read_password
import pandas as pd

def convert_date_format(date_str):
    if isinstance(date_str, datetime):
        # print("date_str 是 datetime.datetime 对象")
        # 如果是 datetime.datetime 对象，则可以在这里执行转换为字符串的操作
        date_str = date_str.strftime('%Y-%m-%d')
        # print("转换后的日期字符串: ", formatted_date_str)
    # else:
    #     print("date_str 不是 datetime.datetime 对象")
    try:
        # 提取年份、月份和日期
        year, month, day = map(int, date_str.split('-'))
        # 修正月份和年份
        if month > 12:
            corrected_month = month % 12
            corrected_year = year + month // 12
        else:
            corrected_month = month
            corrected_year = year
            # 创建新的日期对象
        corrected_date = datetime(corrected_year, corrected_month, day)
        new_date_string = corrected_date.strftime('%Y-%m-%d')
        return new_date_string
    except Exception as e:
        print(f'Invalid date format, please use YYYY-MM-DD:{date_str}', e)
        return None


class YM_order_listing:
    def __init__(self, start_date, end_date=None, seller_sku=None, asin=None) -> None:
        if end_date is None:
            end_date = datetime.today().strftime('%Y-%m-%d')
        # 时间格式设置为YYYY-MM-DD
        # start_date拆分成年月日
        self.start_date = convert_date_format(start_date)
        self.end_date = convert_date_format(end_date)
        if self.start_date is None or self.end_date is None:
            raise ValueError("Invalid date format, please use YYYY-MM-DD")
        self.year, self.month, self.day = self.start_date.split(
            '-')  # type: ignore
        self.result = None
        self.seller_sku = seller_sku
        self.client = Client(host='121.37.249.98', port='9001', user='jinji',
                             password='jifnjiiang_10d6s37', database='yibai_dcm_order_sync')
        self.duck_order = GetFromDuckdb(r"E:\duckdb_database/", 'ym_order')
        self.asin = asin
        if int(self.year) == datetime.today().year:
            self.year_list = ['', '_archtive']
        else:
            self.year_list = ['', '_archtive',
                              '_2020', '_2021', '_2022', '_2023']
        self.warehouse_info = warehouse_info()
        # print(self.warehouse_info)
        self.warehouse_info_id = self.warehouse_info[self.warehouse_info['type'].isin(
            ('third', 'overseas'))]['warehouse_id'].unique().tolist()

    def exe_order_sql(self, function):
        sql_ck = function()
        try:
            df_ck = ck_read(self.client, sql_ck)
            if not df_ck.empty:
                self.result = df_ck.dropna()
            else:
                print('数据为空')
        except Exception as e:
            print(e)
            # 输出错误信息
        return self.result

    def ym_order_sql(self):
        """生成订单查询的sql语句"""
        sql_ck = f"""with '{self.start_date}' as start_day,
            '{self.end_date}' as end_day,
            order_list as (
                select
                    *
                from
                    yibai_dcm_order_sync.dcm_order
                where
                    purchase_time >= start_day
                    and purchase_time < end_day 
                    -- and platform_code not in ('TTS') --剔除tiktok的订单
                    and order_status in (30, 50, 60, 70)
                    and order_type in (1, 2, 3)
                    and platform_status != 'Canceled'
                    and payment_status = 1
                    and is_abnormal = 0
                    and is_intercept = 0
                    and refund_status in (0, 2)
                    and total_price != 0
                    and match(order_id, '-RE') = 0
                    and warehouse_id in (
                        select
                            id
                        from
                            yb_datacenter.yb_warehouse
                        where
                            (
                                `type` IN ('third', 'overseas')
                                AND enabled IN (1)
                                and country not in ('CN')
                            )
                            or id in(481, 854)
                    ) -- union all 
            ),
            warehouse_df as (
                select
                    whid.*
                except
            (real_warehouse_id),
                    yws.name as real_warehouse_name
                from
                    (
                        select
                            id as warehouse_id,
                            name as warehouse_name,
                            `type` as warehouse_type,
                            code as warehouse_code,
                            real_warehouse_id,
                            country as warehouse_country
                        from
                            yb_datacenter.yb_warehouse
                    ) whid
                    left join yb_datacenter.yb_warehouse yws on whid.real_warehouse_id = yws.id
            ),
            order_list_df_with_warehouse_df as (
                select
                    oldf.*,
                    wd.*
                except
            (warehouse_id)
                from
                    order_list oldf
                    inner join warehouse_df wd on oldf.warehouse_id = wd.warehouse_id
            ),
            order_detail_df as (
                select
                    *
                from
                    yibai_dcm_order_sync.dcm_order_detail
                where
                    order_id in (
                        select
                            order_id
                        from
                            order_list_df_with_warehouse_df
                    )
            ),
            order_sku_df as (
                select
                    *
                from
                    yibai_dcm_order_sync.dcm_order_sku
                where
                    order_id in (
                        select
                            order_id
                        from
                            order_list_df_with_warehouse_df
                    )
            ),
            order_profit_df as (
                select
                    order_id,
                    profit,
                    profit_rate,
                    true_profit_rate as true_profit_rate_new1
                from
                    yibai_dcm_order_sync.dcm_order_profit
                where
                    order_id in (
                        select
                            order_id
                        from
                            order_list_df_with_warehouse_df
                    )
            )
            select
                a.order_id as order_id,
                a.platform_order_id as platform_order_id,
                a.account_id as account_id,
                b.seller_sku as seller_sku,
                c.sku as sku,
                b.platform_code as platform_code,
                c.quantity as quantity,
                a.purchase_time as purchase_time,
                date(a.payment_time) as update_time,
                a.payment_time as payment_time,
                a.ship_country as ship_country,
                a.ship_country_name as ship_country_name,
                a.warehouse_id as warehouse_id,
                a.is_ship_process as is_ship_process,
                a.platform_status as platform_status,
                a.total_price as total_price,
                a.warehouse_country as warehouse_country,
                a.warehouse_name as warehouse_name,
                a.warehouse_type as warehouse_type,
                a.warehouse_code as warehouse_code,
                a.real_warehouse_name as real_warehouse_name,
                d.profit as profit,
                if(
                    ifNull(d.true_profit_rate_new1, '0') = '0',
                    d.profit_rate,
                    d.true_profit_rate_new1
                ) as profit_rate
            from
                order_list_df_with_warehouse_df a
                right join order_detail_df b on a.order_id = b.order_id
                right join order_sku_df c on b.id = c.order_detail_id
                right join order_profit_df d on a.order_id = d.order_id;"""
        # print(sql_ck)
        return sql_ck

    def month_order(self):
        """根据时间拉取销量"""
        self.result = self.exe_order_sql(function=self.ym_order_sql)
        self.result = self.result[self.result['profit']!='999999.9999']
        self.duck_order.to_parquet_hive(
            self.result, f"ym_order", partition_cols='update_time')
        return self.result


def main():
    date_generator = DateGenerator('2024-09-01', '2024-12-25')
    for year, month in date_generator._generate_dates():
        print(year, month, end='\t')
        start_date = datetime.strptime(f'{year}-{month}-01', '%Y-%m-%d')
        end_date = start_date + relativedelta(months=1)
        ym_order = YM_order_listing(start_date=start_date, end_date=end_date)
        ym_order.month_order()
    print("订单信息已生成")


# if __name__ == '__main__':
    # ym_order = YM_order_listing(start_date='2024-12-01', end_date='2024-12-10')
    # ym_order.month_order()
    # main()


In [23]:
df['时间'] = df['update_time'].dt.strftime('%Y-%m')

In [20]:
df.to_excel(r'C:\Users\Administrator\Desktop\易佰23年1月到24年海外仓订单.xlsx',index=False)

In [16]:
duck = GetFromDuckdb(database='mrp_oversea')
sql = f"""select sku ,ship_country,sum(ifnull(sku_quantity::int,0)) as sku_quantity_1_4,update_time
from read_parquet('E:/duck/parquet_file/yb_oversea_order/*/*.parquet', union_by_name=true) 
where update_time >='2023-01-01' and update_time <'2024-01-01' 
group by update_time,sku ,ship_country"""
df = duck.exe_duckdb(sql)
# df.to_excel(r'C:\Users\Administrator\Desktop\易佰23年1月到24年海外仓订单.xlsx',index=False)

In [17]:
df.rename(columns={'sku': 'SKU'}, inplace=True)

In [18]:
from math import ceil 
from tqdm import tqdm
def product_sku(df):
    """SKU信息：最新采购价(元),最小起订量,整箱箱率,外箱尺寸(cm),整箱重量(kg),外箱体积(cm³)"""
    df['SKU'] = df['SKU'].astype(str).str.replace(r'\((.*)|\s*|\t*', '')
    sku_list = df['SKU'].dropna().unique().tolist()
    num = 2000
    count = ceil(len(sku_list) / num)
    df_purchase_data = pd.DataFrame()
    for i in tqdm(range(count)):
        sql9 = f'''SELECT sku AS SKU, last_price AS "最新采购价(元)",
        case 
            when is_drawback = 0 then '否' else '是' 
        end AS "是否可退税",
        500/(tax_rate/100)*(1+ticketed_point/100)/last_price  as "最小退税标准" ,
        ticketed_point/100 as "开票点",
        case when maintain_ticketed_point = 0 then '没有维护' else '有维护' end AS "是否维护开票点",
        starting_qty AS "最小起订量", supplier_code AS "供应商编码",inside_number AS "整箱箱率",product_name as "商品名称",
        box_size AS "外箱尺寸(cm)", outer_box_volume AS "外箱体积(cm³)",
        sample_package_length as "产品包装长(cm)", sample_package_width "产品包装宽(cm)", sample_package_heigth "产品包装高(cm)", sample_package_weight "产品包装重量(g)" ,
        sample_package_length*sample_package_width*sample_package_heigth/1000000 "产品体积(m³)"
        FROM yibai_purchase_sync.pur_product
        WHERE sku in ({", ".join(["'{}'".format(sku) for sku in sku_list[i*num:(i+1)*num]])}) '''
        purchase_data_0 = read_password(84, sql_name=sql9)
        df_purchase_data = pd.concat(
            [df_purchase_data, purchase_data_0], axis=0)
    df_purchase_data['SKU'] = df_purchase_data['SKU'].astype(str).str.replace(
        r'\((.*)|\s*|\t*', '')
    df_purchase_data = df_purchase_data.drop_duplicates()
    print(df_purchase_data.info())
    df_purchase_data.loc[df_purchase_data['整箱箱率'] == 0, '整箱箱率'] = 1
    df_purchase_data.loc[df_purchase_data['整箱箱率'] == '无', '整箱箱率'] = 1
    df_purchase_data.loc[df_purchase_data['整箱箱率'].isnull(), '整箱箱率'] = 1
    df_purchase_data.loc[df_purchase_data['最小起订量'] == 0, '最小起订量'] = 1
    df_purchase_data.loc[df_purchase_data['最小起订量'] == '无', '最小起订量'] = 1
    df_purchase_data.loc[df_purchase_data['最小起订量'].isnull(), '最小起订量'] = 1
    df_purchase_data =df_purchase_data[['最新采购价_元_','SKU']]
    df = df.merge(df_purchase_data, on='SKU', how='left')
    return df

df = product_sku(df)


100%|██████████| 6/6 [00:01<00:00,  3.63it/s]

<class 'pandas.core.frame.DataFrame'>
Index: 10597 entries, 0 to 734
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SKU        10597 non-null  object 
 1   最新采购价_元_   10597 non-null  object 
 2   是否可退税      10597 non-null  object 
 3   最小退税标准     10597 non-null  float64
 4   开票点        10597 non-null  float64
 5   是否维护开票点    10597 non-null  object 
 6   最小起订量      10597 non-null  int64  
 7   供应商编码      10597 non-null  object 
 8   整箱箱率       10597 non-null  int64  
 9   商品名称       10597 non-null  object 
 10  外箱尺寸_cm_   10597 non-null  object 
 11  外箱体积_cm³_  10597 non-null  float64
 12  产品包装长_cm_  10597 non-null  float64
 13  产品包装宽_cm_  10597 non-null  float64
 14  产品包装高_cm_  10597 non-null  float64
 15  产品包装重量_g_  10597 non-null  float64
 16  产品体积_m³_   10597 non-null  float64
dtypes: float64(8), int64(2), object(7)
memory usage: 1.5+ MB
None


In [13]:
df

,SKU,ship_country,sku_quantity_1_4,update_time,最新采购价_元_
0,1613210005711,PH,8.0,2024-01-01,7.8
1,JM01384-01,PH,4.0,2024-01-01,7
2,JM02681,PH,1.0,2024-01-01,26.01
3,QC17698-03,PH,3.0,2024-01-01,5.5
4,GS00951,PH,1.0,2024-01-01,5.5
...,...,...,...,...,...
130952,GS04339,RU,2.0,2024-04-30,26.5
130953,2613210036411,RU,1.0,2024-04-30,25
130954,1613230007711,RU,1.0,2024-04-30,36.09
130955,1613220012711,RU,1.0,2024-04-30,40.02


In [5]:
df = pd.read_excel(r'C:\\Users\\Administrator\\Desktop\\开发来源.xlsx')
df_1 = read_password(81,sql_name=f"""
	SELECT
		sku ,
		case 
			when develop_source = 14 then '通拓'
			when develop_source = 15 then '通拓'
			when develop_source = 22 then '通拓'
			else '易佰'
		end as "开发类型"
	from yibai_prod_base_sync.yibai_prod_sku;""")
df = df.merge(df_1 ,how = 'left',on = 'sku')

In [7]:
df.to_excel(r'C:\\Users\\Administrator\\Desktop\\开发来源.xlsx')

In [163]:
sql = f"""SELECT sku as SKU,warehouse_id ,available_stock as "可用库存" FROM yb_stock_center.yb_stock x
    WHERE cargo_owner_id =8  and available_stock>0"""
df_warehouse = read_password(63, sql)
df_warehouse_copy = df_warehouse.rename(
        columns={'可用库存': '虚拟仓库存'})

In [111]:
df_warehouse_copy = df_warehouse_copy.merge(warehouse, how='left',on='warehouse_id')
df_warehouse_copy = df_warehouse_copy[~df_warehouse_copy['warehouse_name'].isna()]
df_warehouse_copy = df_warehouse_copy[df_warehouse_copy['warehouse_name'].str.contains('Temu')]

In [120]:
from  header_files.func import root_window_path
file = root_window_path()
df = pd.read_excel(file)

In [127]:
df = df.rename(columns={'原仓': 'warehouse_name'})

In [134]:
df_1.to_excel('C:/Users/Administrator/Desktop/独享仓调拨.xlsx')

In [136]:
df_1['虚拟仓库存'] = df_1['虚拟仓库存'].fillna(0)

In [141]:
df_1 = df_1[df_1['虚拟仓库存']<=1]

In [154]:
df_1 =df_1.merge(warehouse[['warehouse_name','real_warehouse_name']],how='left',on='warehouse_name')

In [171]:
df_1 = df_1.merge(df_warehouse_copy_real[['real_warehouse_name','SKU','虚拟仓库存']],how='left',on=['SKU','real_warehouse_name'])

In [167]:
df_warehouse_copy_real = df_warehouse_copy[df_warehouse_copy['real_warehouse_name']==df_warehouse_copy['warehouse_name']]

In [179]:
df_warehouse_copy[df_warehouse_copy['type'].isin(['third'])].to_excel('C:/Users/Administrator/Desktop/独享仓调拨11.xlsx')

In [133]:
df_1 = df.merge(df_warehouse_copy,how='left',on=['SKU','warehouse_name'])

In [3]:
sql_four_stock = f"""SELECT sku,destination_warehouse,"采购在途数量","中转仓库存数量","海外仓在途数量","海外仓库存数量" 
FROM yibai_mrp_oversea.yibai_oversea_four_stock_all_20241210 x
WHERE "采购在途数量"+"中转仓库存数量"+"海外仓在途数量"+"海外仓库存数量">0 AND destination_warehouse not in ('DE8','RU6','ST6')"""
df = read_password(id_int=71,sql_name=sql_four_stock)

In [6]:
duck = GetFromDuckdb(database='mrp_oversea')

In [12]:
from pandas import DataFrame
import yaml 

def warehouse_info() -> DataFrame:
    duck = GetFromDuckdb(database='oversea')
    df_temp = read_password(60, f"""with ware as (SELECT x.* FROM yb_stock_center.yb_warehouse x),
            house as (
            SELECT x.id as real_warehouse_id, name as real_warehouse_name,code as real_warehouse_code FROM yb_stock_center.yb_warehouse x
            WHERE real_warehouse_id = id)
            SELECT wa.id as warehouse_id , wa.name as warehouse_name ,wa.code as warehouse_code,wa.type,
            hs.real_warehouse_id,hs.real_warehouse_name as real_warehouse_name,hs.real_warehouse_code as real_warehouse_code,
            enabled,country,create_time
            from ware wa left join house hs on wa.real_warehouse_id = hs.real_warehouse_id;""")
    with open(r'D:\VScode\learn-git\my-repo\header_files\country_codes.yaml', 'r', encoding='utf-8') as stream:
        data = yaml.safe_load(stream)  # 只调用一次，并将结果存储在 data 变量中
        country_code_to_name = data.get(
            'country_code_to_name', None)  # 使用 get 方法，避免 KeyError
    df_temp['warehouse'] = df_temp['country'].map(country_code_to_name)
    df_temp['warehouse'] = df_temp['warehouse'] + '仓'
    duck.to_duckdb(df_temp, 'yb_warehouse', if_exists='replace')
    df_temp = df_temp[df_temp['enabled'] == 1]
    return df_temp

warehouse = warehouse_info()

In [16]:
df = df.merge(warehouse, how='left', left_on='destination_warehouse',right_on = 'warehouse_name')
df.loc[df['destination_warehouse'] == 'XYD菲律宾海外仓', 'warehouse'] = '菲律宾仓'
df.loc[df['destination_warehouse'] == 'XYD马来海外仓', 'warehouse'] = '马来西亚仓'

In [27]:
from datetime import datetime,timedelta
import pandas as pd
from header_files.func import read_password, root_window_path, time_wrapper
from header_files.pandas_sql import GetFromDuckdb,DateGenerator
from header_files.orderlisting import OrderListing
import pymysql
from tqdm import tqdm
import yaml
from math import ceil

@time_wrapper
def product_sku(df):
    """SKU信息：最新采购价(元),最小起订量,整箱箱率,外箱尺寸(cm),整箱重量(kg),外箱体积(cm³)"""
    df['sku'] = df['sku'].astype(str).str.replace(r'\((.*)|\s*|\t*', '')
    conn6 = pymysql.connect(host='121.37.228.71',
                            port=9030,
                            user='yibai208384',
                            password='XjcT3ImVrb',
                            database='yibai_purchase')
    sku_list = df['sku'].dropna().unique().tolist()
    num = 2000
    count = ceil(len(sku_list) / num)
    df_purchase_data = pd.DataFrame()
    for i in tqdm(range(count)):
        sql9 = f'''SELECT sku, last_price AS "最新采购价(元)"
        FROM yibai_purchase.pur_product
        WHERE sku in ({", ".join(["'{}'".format(sku) for sku in sku_list[i*num:(i+1)*num]])}) '''
        purchase_data_0 = pd.read_sql(sql9, conn6)
        df_purchase_data = pd.concat(
            [df_purchase_data, purchase_data_0], axis=0)
    df_purchase_data['sku'] = df_purchase_data['sku'].astype(str).str.replace(
        r'\((.*)|\s*|\t*', '')
    df_purchase_data = df_purchase_data.drop_duplicates()
    # df_purchase_data =df_purchase_data[['商品名称','SKU']]
    df = df.merge(df_purchase_data, on='sku', how='left')
    return df

df= product_sku(df)

df_1 = read_password(81,sql_name=f"""
	SELECT
		sku ,
		case 
			when develop_source = 14 then '通拓'
			when develop_source = 15 then '通拓'
			when develop_source = 22 then '通拓'
			else '易佰'
		end as "开发类型"
	from yibai_prod_base_sync.yibai_prod_sku;""")
df = df.merge(df_1 ,how = 'left',on = 'sku')
df['最新采购价(元)'] = df['最新采购价(元)'].fillna(40)


100%|██████████| 41/41 [00:09<00:00,  4.39it/s]


product_sku函数执行时间为:00:00:09


In [93]:
sql = f"""WITH aa AS (select order_id,platform_order_id ,account_id ,seller_sku ,sku ,platform_code ,quantity ,purchase_time,
    warehouse_id ,total_price ,profit ,profit_rate,
    toFloat64(ifNull(last_price,0)) as last_price
from yibai_mrp_oversea.yibai_oversea_orders_statistics_temp_all_20241210 a
left join yibai_purchase_sync.pur_product b on a.sku=b.sku
union all
select order_id,platform_order_id ,account_id ,seller_sku ,sku ,platform_code ,quantity ,purchase_time,
    warehouse_id ,total_price ,toString(profit) as profit ,toString(profit_rate) as profit_rate,toFloat64(ifNull(new_price,0)) as last_price
from yibai_mrp_oversea.yibai_oversea_dcm_orders_statistics_temp_all_20241210 a
left join yibai_dcm_base_sync.dcm_product b on a.sku=b.sku
union all
select order_id,platform_order_id ,account_id ,seller_sku ,sku ,platform_code ,quantity ,toDateTime(purchase_time) purchase_time,
    warehouse_id ,total_price ,toString(profit) as profit ,toString(profit_rate) as profit_rate,last_price
from yibai_mrp_oversea.tt_orders_20241210)
SELECT sku,SUM( quantity*last_price) AS "销库金额" 
FROM aa 
WHERE purchase_time>'2024-11-10'
GROUP BY sku;"""

In [94]:
df = read_password(id_int=75,sql_name=sql)
df_1 = read_password(81,sql_name=f"""
	SELECT
		sku ,
		case 
			when develop_source = 14 then '通拓'
			when develop_source = 15 then '通拓'
			when develop_source = 22 then '通拓'
			else '易佰'
		end as "开发类型"
	from yibai_prod_base_sync.yibai_prod_sku;""")
df = df.merge(df_1 ,how = 'left',on = 'sku')

In [97]:
df['开发类型'] = df['开发类型'].fillna('易佰')

In [98]:
df

,sku,销库金额,开发类型
0,GS07402,336.0,易佰
1,JYB00242,221.0,易佰
2,I3383,1781.9,通拓
3,10326072,52.0,易佰
4,W11199EU,196.5,通拓
...,...,...,...
74893,1618200341711,175.0,易佰
74894,2720220351111,100.0,易佰
74895,1511200054011,32.0,易佰
74896,YM139L2-2873062-YYR,120.0,易佰


In [34]:
df['四段库存数量'] = df['采购在途数量']+ df['中转仓库存数量']+df['海外仓在途数量']+df['海外仓库存数量']

In [37]:
for i in('采购在途数量', '中转仓库存数量', '海外仓在途数量','海外仓库存数量','四段库存数量'):
    i_name =i.replace('数量', '金额(万元)')
    df[i_name]= df[i]*df['最新采购价(元)']/10000

In [42]:
df_tt_yb = df.groupby(by = ['开发类型','sku','warehouse']).agg({'采购在途数量':'sum', '中转仓库存数量':'sum', '海外仓在途数量':'sum','海外仓库存数量':'sum','四段库存数量':'sum','四段库存数量':'sum', '采购在途金额(万元)':'sum', '中转仓库存金额(万元)':'sum', '海外仓在途金额(万元)':'sum', '海外仓库存金额(万元)':'sum','四段库存金额(万元)':'sum'}).reset_index()

In [45]:
df_tt_yb['warehouse'] = df_tt_yb['warehouse'].replace('捷克仓|德国仓|法国仓|意大利仓|西班牙仓','欧洲仓',regex=True)

In [47]:
df_tt_yb = df_tt_yb.groupby(by = ['开发类型','sku','warehouse']).agg({'采购在途数量':'sum', '中转仓库存数量':'sum', '海外仓在途数量':'sum','海外仓库存数量':'sum','四段库存数量':'sum','四段库存数量':'sum', '采购在途金额(万元)':'sum', '中转仓库存金额(万元)':'sum', '海外仓在途金额(万元)':'sum', '海外仓库存金额(万元)':'sum','四段库存金额(万元)':'sum'}).reset_index()

In [51]:
sql_ = f"""SELECT sku,site,round(SUM(quantity),0) as quantity 
FROM yibai_mrp_oversea.yibai_oversea_orders_total_all_20241210 x
where real_rate>0 and created_time >='2024-11-10' AND created_time <'2024-12-10'
group by sku,site;"""


df = read_password(id_int=71,sql_name=sql_)


In [81]:
df_tt_yb = product_sku(df_tt_yb)
df_tt_yb['最新采购价(元)'] = df_tt_yb['最新采购价(元)'].fillna(40)

100%|██████████| 41/41 [00:15<00:00,  2.61it/s]


product_sku函数执行时间为:00:00:16


In [61]:
df_tt_yb['国家'] = df_tt_yb['warehouse'].str.replace('仓', '', regex=True)
df_tt_yb =df_tt_yb.merge(df[['国家','sku','quantity']], how='left',on=['国家','sku'])
df_tt_yb['quantity'] = df_tt_yb['quantity'].fillna(0)

In [76]:
df['最新采购价(元)'] = df['最新采购价(元)'].fillna(40)
df['正净利销库金额'] = df['quantity'] * df['最新采购价(元)']

In [56]:
with open(r'D:\VScode\learn-git\my-repo\header_files\country_codes.yaml', 'r', encoding='utf-8') as stream:
    data = yaml.safe_load(stream)  # 只调用一次，并将结果存储在 data 变量中
    country_code_to_name = data.get(
        'country_code_to_name', None) 
df['国家'] = df['site'].str.upper().map(country_code_to_name)
site = '德国|法国|奥地利|爱尔兰|西班牙|比利时|保加利亚|荷兰|捷克|斯洛伐克|意大利|匈牙利|挪威|葡萄牙|瑞典|波兰|瑞士|斯洛文尼亚|卢森堡|克罗地亚|希腊|马耳他|芬兰|拉脱维亚|塞浦路斯|罗马尼亚|丹麦|土耳其'
df['国家'] = df['国家'].str.replace(site , '欧洲',regex=True)
df['国家'] = df['国家'].str.replace('新西兰|澳大利亚' , '澳洲',regex=True)
df = df.groupby(by=['国家','sku']).agg({'quantity':'sum'}).reset_index()
df = product_sku(df)
df = df.merge(df_1 ,how = 'left',on = 'sku')

100%|██████████| 30/30 [00:11<00:00,  2.62it/s]


product_sku函数执行时间为:00:00:11


In [4]:
duck = GetFromDuckdb(database='mrp_oversea')
sql = f"""select * from read_parquet('E:/duck/parquet_file/*_order/*/*.parquet', union_by_name=true,hive_partitioning=True
)where update_time >= '2024-11-19' and update_time < '2024-12-20' and profit_rate >='0.28'"""
df = duck.exe_duckdb(sql)

In [55]:
df.loc[df['quantity'].isna(),'quantity']=df.loc[df['quantity'].isna(),'sku_quantity']
df_date = df[['sku','update_time','ship_country','quantity']]
df_date['quantity'] = df_date['quantity'].astype(int)
df_date['ship_country'] = df_date['ship_country'].str.upper()
df_date['ship_country'] = df_date['ship_country'].replace('EAST','US',regex=True)
df_date['ship_country'] = df_date['ship_country'].replace('SYD','AU',regex=True)
df_date = df_date.groupby(['sku','update_time','ship_country']).agg({'quantity':'sum'}).reset_index()

In [56]:
import yaml
with open(r'D:\VScode\learn-git\my-repo\header_files\country_codes.yaml', 'r', encoding='utf-8') as stream:
    data = yaml.safe_load(stream)  # 只调用一次，并将结果存储在 data 变量中
    country_code_to_name = data.get(
        'country_code_to_name', None) 
df_date['国家'] = df_date['ship_country'].str.upper().map(country_code_to_name)

In [59]:
site = '德国|法国|奥地利|爱尔兰|西班牙|比利时|保加利亚|荷兰|捷克|斯洛伐克|意大利|匈牙利|挪威|葡萄牙|瑞典|波兰|瑞士|斯洛文尼亚|卢森堡|克罗地亚|希腊|马耳他|芬兰|拉脱维亚|塞浦路斯|罗马尼亚|丹麦|土耳其|爱沙尼亚'
df_date['国家'] = df_date['国家'].str.replace(site , '欧洲',regex=True)
df_date['国家'] = df_date['国家'].str.replace('新西兰|澳大利亚' , '澳洲',regex=True)

In [63]:
df_date = df_date.groupby(['sku','update_time','国家']).agg({'quantity':'sum'}).reset_index().sort_values('update_time',ascending=False)

In [62]:
duck.to_duckdb(df_date,table_name='yb_tt_ym_order')

In [71]:
df_date['sort_id'] = df_date.groupby(['sku', '国家'])[
    'update_time'].rank(method='first', ascending=False)
today = datetime.now().strftime("%Y%m%d")
duck.to_duckdb(df=df_date, table_name=f'ever_order_sort_{today}')

In [74]:
duck.__dict__

{'file': WindowsPath('E:/duckdb_database'),
 'db_name': 'E:\\duckdb_database\\mrp_oversea.duckdb',
 'file_save': WindowsPath('E:/duck/parquet_file'),
 'tesk_dir': WindowsPath('D:/Desktop'),
 'table': 'ever_order_sort_20241219'}

In [77]:

sql = f"""SELECT sku as"SKU","国家", 
    sum(case when update_time >=today()-30 and sort_id<=7 then quantity else 0 end)/7 as "近7日正净利日均销量",
    sum(case when update_time >=today()-30 then quantity else 0 end) as"近30天正净利销量总和"
    FROM mrp_oversea.main.ever_order_sort_{today}
    group by "国家",sku"""
df_sale_order = duck.exe_duckdb(sql)
#data = data.merge(df_sale_order, on=['SKU', '国家'], how='left')

In [80]:
df[df['sku'] == '3114230100111']

,order_id,platform_order_id,account_id,seller_sku,sku,platform_code,quantity,purchase_time,payment_time,update_time,...,true_profit_new1,sku_quantity,sales_status,warehouse_name,warehouse_code,type,warehouse,warehouse_country,warehouse_type,real_warehouse_name


In [78]:
df_sale_order.to_excel(r'C:\\Users\\Administrator\\Desktop\\销量20241219.xlsx')

In [111]:
sql = f"""with sh as (SELECT sku,case 
	when lower(ship_country) = 'east' then 'us' 
	when lower(ship_country) = 'syd' then 'au' 
	else lower(ship_country)
end as ship_country ,
sum(CASE when quantity is null then sku_quantity else quantity::int end) as quantity,
SUM(CASE WHEN profit_rate::float > 0.16  THEN CASE when quantity is null then sku_quantity else quantity::int end::int ELSE 0 END) AS "正净利订单"
FROM ym_order.main.yb_tt_ym_order x
group by sku,ship_country)
select sku,ship_country,SUM(quantity) as quantity,SUM("正净利订单") as "正净利订单销库" 
from sh
group by sku,ship_country;"""
df = duck.exe_duckdb(sql=sql)

In [112]:
from datetime import datetime, timedelta
import pandas as pd
from header_files.func import read_password, root_window_path, time_wrapper
from header_files.pandas_sql import GetFromDuckdb, DateGenerator
from header_files.orderlisting import OrderListing
import pymysql
from tqdm import tqdm
import yaml
from math import ceil
import warnings

@time_wrapper
def product_sku(df):
    """SKU信息：最新采购价(元),最小起订量,整箱箱率,外箱尺寸(cm),整箱重量(kg),外箱体积(cm³)"""
    df['sku'] = df['sku'].astype(str).str.replace(r'\((.*)|\s*|\t*', '')
    conn6 = pymysql.connect(host='121.37.228.71',
                            port=9030,
                            user='yibai208384',
                            password='XjcT3ImVrb',
                            database='yibai_purchase')
    sku_list = df['sku'].dropna().unique().tolist()
    num = 2000
    count = ceil(len(sku_list) / num)
    df_purchase_data = pd.DataFrame()
    for i in tqdm(range(count)):
        sql9 = f'''SELECT sku , last_price AS "最新采购价(元)", starting_qty AS "最小起订量", supplier_code AS "供应商编码",inside_number AS '整箱箱率',product_name2 as "商品名称",
        box_size AS '外箱尺寸(cm)', box_weight AS '整箱重量(kg)', outer_box_volume AS '外箱体积(cm³)',
        sample_package_length as "产品包装长(cm)", sample_package_width "产品包装宽(cm)", sample_package_heigth "产品包装高(cm)", sample_package_weight "产品包装重量(g)" ,
        sample_package_length*sample_package_width*sample_package_heigth/1000000 '产品体积(m³)'
        FROM yibai_purchase.pur_product
        WHERE sku in ({", ".join(["'{}'".format(sku) for sku in sku_list[i*num:(i+1)*num]])}) '''
        purchase_data_0 = pd.read_sql(sql9, conn6)
        df_purchase_data = pd.concat(
            [df_purchase_data, purchase_data_0], axis=0)
    df_purchase_data['sku'] = df_purchase_data['sku'].astype(str).str.replace(
        r'\((.*)|\s*|\t*', '')
    df_purchase_data = df_purchase_data.drop_duplicates()
    # df_purchase_data =df_purchase_data[['商品名称','SKU']]
    df = df.merge(df_purchase_data, on='sku', how='left')
    return df

df_1 = read_password(81,sql_name=f"""
	SELECT
		sku ,
		case 
			when develop_source = 14 then '通拓'
			when develop_source = 15 then '通拓'
			when develop_source = 22 then '通拓'
			else '易佰'
		end as "开发类型"
	from yibai_prod_base_sync.yibai_prod_sku;""")


In [113]:
with open(r'D:\VScode\learn-git\my-repo\header_files\country_codes.yaml', 'r', encoding='utf-8') as stream:
    data = yaml.safe_load(stream)  # 只调用一次，并将结果存储在 data 变量中
    country_code_to_name = data.get(
        'country_code_to_name', None) 
df['国家'] = df['ship_country'].str.upper().map(country_code_to_name)
site = '德国|法国|奥地利|爱尔兰|西班牙|比利时|保加利亚|荷兰|捷克|斯洛伐克|意大利|匈牙利|挪威|葡萄牙|瑞典|波兰|瑞士|斯洛文尼亚|卢森堡|克罗地亚|希腊|马耳他|芬兰|拉脱维亚|塞浦路斯|罗马尼亚|丹麦|土耳其'
df['国家'] = df['国家'].str.replace(site , '欧洲',regex=True)
df['国家'] = df['国家'].str.replace('新西兰|澳大利亚' , '澳洲',regex=True)
df = df.groupby(by=['国家','sku']).agg({'quantity':'sum','正净利订单销库':'sum'}).reset_index()
df = product_sku(df)
df = df.merge(df_1 ,how = 'left',on = 'sku')

100%|██████████| 28/28 [00:13<00:00,  2.00it/s]


product_sku函数执行时间为:00:00:14


In [114]:
duck.to_duckdb(df,table_name='overseas_order_price')

In [115]:
df.to_excel(f'C:\\Users\\Administrator\\Desktop\\11月海外仓正净利销库.xlsx',index=False)